# CNN via Keras

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, read_csv, get_dummies

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

print (keras.__version__)

Using TensorFlow backend.


2.0.4


In [2]:
train = read_csv('train.csv')
test = read_csv('test.csv')

In [17]:
SAMPLE_SIZE = 1000#42000
NUM_CLASSES = 10
IMG_WIDTH = 28
IMG_HEIGHT = 28
IMG_CHANNEL = 1

X_all = train.iloc[:SAMPLE_SIZE, 1:].copy()
y_all = train.iloc[:SAMPLE_SIZE, 0].copy()
X_test = test.copy()

X_all = X_all.astype('float32') / 255
X_test = X_test.astype('float32') / 255

X_all = X_all.values.reshape(-1, IMG_WIDTH, IMG_HEIGHT, IMG_CHANNEL)
X_test = X_test.values.reshape(-1, IMG_WIDTH, IMG_HEIGHT, IMG_CHANNEL)
y_all = get_dummies(y_all.values).values

# Graph

In [ ]:
m1 = Sequential()
m1.add(Conv2D(filters=32, kernel_size=(5, 5), strides=1, padding='SAME',
             activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, IMG_CHANNEL)))
m1.add(Flatten())
m1.add(Dropout(0.3))
m1.add(Dense(NUM_CLASSES, activation='softmax'))
m1.compile(loss='categorical_crossentropy', optimizer='Adadelta', metrics=['accuracy'])

In [ ]:
m2 = Sequential()
m2.add(Conv2D(filters=32, kernel_size=(5, 5), strides=1, padding='SAME',
             activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, IMG_CHANNEL)))
m2.add(Flatten())
m2.add(Dropout(0.3))
m2.add(Dense(128, activation='relu')) # add
m2.add(Dense(NUM_CLASSES, activation='softmax'))
m2.compile(loss='categorical_crossentropy', optimizer='Adadelta', metrics=['accuracy'])

In [55]:
m3 = Sequential()
m3.add(Conv2D(filters=32, kernel_size=(5, 5), strides=1, padding='SAME',
             activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, IMG_CHANNEL)))
m3.add(Conv2D(filters=64, kernel_size=(5, 5), strides=1, padding='SAME',
             activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, IMG_CHANNEL)))
m3.add(MaxPooling2D(pool_size=(2, 2)))
m3.add(Flatten())
m3.add(Dropout(0.3))
m3.add(Dense(128, activation='relu'))#去掉以后变很差
m3.add(Dense(NUM_CLASSES, activation='softmax'))
m3.compile(loss='categorical_crossentropy', optimizer='Adadelta', metrics=['accuracy'])

In [56]:
m4 = Sequential()
m4.add(Conv2D(filters=32, kernel_size=(5, 5), strides=1, padding='SAME',
             activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, IMG_CHANNEL)))
#m4.add(MaxPooling2D(pool_size=(2, 2)))#快了，不过early stopping达到的准确的变小了
m4.add(Conv2D(filters=64, kernel_size=(5, 5), strides=1, padding='SAME',
             activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, IMG_CHANNEL)))
m4.add(MaxPooling2D(pool_size=(2, 2)))

# 没有变更好，有可能是因为数据量不足
m4.add(Conv2D(filters=128, kernel_size=(5, 5), strides=1, padding='SAME',
             activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, IMG_CHANNEL)))
m4.add(Conv2D(filters=256, kernel_size=(5, 5), strides=1, padding='SAME',
             activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, IMG_CHANNEL)))
m4.add(MaxPooling2D(pool_size=(2, 2)))

m4.add(Flatten())
m4.add(Dropout(0.3))
#m4.add(Dense(1024, activation='relu'))#慢些了（好理解，因为复杂），准确度没变（因为conv提取特征没变，而分类已经到极限）
m4.add(Dense(128, activation='relu'))
m4.add(Dense(NUM_CLASSES, activation='softmax'))
m4.compile(loss='categorical_crossentropy', optimizer='Adadelta', metrics=['accuracy'])

# Fit

In [57]:
MODEL_PATH = './model'
EPOCHS = 32
BATCH_SIZE = 128

tb3 = TensorBoard(log_dir=MODEL_PATH+'/m3/', write_images=True)
tb4 = TensorBoard(log_dir=MODEL_PATH+'/m4/', write_images=True)

cp3 = ModelCheckpoint(filepath=MODEL_PATH+"/m3/checkpoint-{epoch:02d}-{val_acc:.2f}.hdf5", 
                      monitor='val_acc', verbose=1, save_best_only=True, mode='max')
cp4 = ModelCheckpoint(filepath=MODEL_PATH+"/m4/checkpoint-{epoch:02d}-{val_acc:.2f}.hdf5", 
                      monitor='val_acc', verbose=1, save_best_only=True, mode='max')

early_stopping = EarlyStopping(monitor='val_loss', patience=1, verbose=1)

print ('\n*****train on model 3*****\n')
his3 = m3.fit(X_all, y_all, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.3, verbose=0,
                  callbacks=[tb3, cp3, early_stopping])
print ('\n*****train on model 4*****\n')
his4 = m4.fit(X_all, y_all, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.3, verbose=0,
                  callbacks=[tb4, cp4, early_stopping])


*****train on model 3*****

Epoch 00000: val_acc improved from -inf to 0.34333, saving model to ./model/m3/checkpoint-00-0.34.hdf5
Epoch 00001: val_acc improved from 0.34333 to 0.52333, saving model to ./model/m3/checkpoint-01-0.52.hdf5
Epoch 00002: val_acc improved from 0.52333 to 0.71000, saving model to ./model/m3/checkpoint-02-0.71.hdf5
Epoch 00003: val_acc did not improve
Epoch 00004: val_acc improved from 0.71000 to 0.79000, saving model to ./model/m3/checkpoint-04-0.79.hdf5
Epoch 00005: val_acc improved from 0.79000 to 0.79333, saving model to ./model/m3/checkpoint-05-0.79.hdf5
Epoch 00006: val_acc did not improve
Epoch 00006: early stopping

*****train on model 4*****

Epoch 00000: val_acc improved from -inf to 0.26333, saving model to ./model/m4/checkpoint-00-0.26.hdf5
Epoch 00001: val_acc improved from 0.26333 to 0.58333, saving model to ./model/m4/checkpoint-01-0.58.hdf5
Epoch 00002: val_acc did not improve
Epoch 00003: val_acc improved from 0.58333 to 0.73000, saving model

In [23]:
m3.predict_classes(X_test[:BATCH_SIZE], batch_size=BATCH_SIZE)

256/256 [==============================] - 2s


array([2, 0, 9, 2, 2, 7, 0, 3, 0, 3, 5, 7, 4, 0, 4, 5, 2, 1, 9, 0, 9, 1, 1,
       5, 7, 4, 2, 7, 7, 7, 7, 5, 4, 2, 6, 2, 9, 5, 1, 6, 7, 7, 4, 9, 8, 7,
       8, 8, 6, 7, 6, 8, 8, 3, 8, 2, 1, 2, 4, 0, 4, 1, 7, 0, 0, 0, 1, 9, 0,
       1, 6, 5, 8, 8, 2, 8, 3, 9, 2, 3, 5, 4, 1, 0, 9, 2, 4, 3, 6, 7, 2, 0,
       6, 6, 1, 4, 3, 9, 7, 4, 0, 8, 2, 0, 7, 3, 0, 5, 0, 9, 0, 0, 4, 2, 1,
       2, 1, 1, 8, 3, 3, 7, 2, 8, 6, 3, 8, 7, 8, 4, 3, 5, 6, 0, 0, 0, 3, 1,
       3, 6, 4, 3, 4, 5, 5, 8, 7, 7, 2, 8, 4, 3, 5, 6, 5, 3, 7, 5, 7, 8, 3,
       0, 4, 5, 1, 8, 7, 6, 3, 0, 2, 7, 8, 6, 1, 3, 7, 4, 1, 2, 4, 8, 5, 2,
       4, 9, 2, 1, 6, 0, 6, 1, 4, 4, 6, 0, 9, 7, 6, 9, 1, 9, 0, 9, 9, 0, 8,
       4, 6, 2, 0, 9, 3, 6, 7, 2, 1, 6, 3, 4, 2, 3, 1, 0, 2, 8, 4, 6, 1, 0,
       0, 4, 9, 1, 2, 3, 2, 3, 8, 6, 8, 6, 2, 8, 5, 5, 4, 8, 3, 8, 9, 7, 1,
       3, 8, 4])

In [22]:
y_pred = m3.predict_classes(X_test, batch_size=BATCH_SIZE)

np.savetxt('prediction.csv', np.c_[range(1, len(yPred)+1), yPred], 
           delimiter=',', header='ImageId,Label', comments='', fmt='%d')

 6400/28000 [=====>........................] - ETA: 118s

KeyboardInterrupt: 